### ADA BOOST

In [ ]:
from imblearn.over_sampling import ADASYN 

ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_sample(X_fe, real_y)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report

In [ ]:
def printing_Kfold_scores(x_train_data,y_train_data):
    fold = KFold(5,shuffle=False)

    # Different C parameters
    c_param_range = [0.01,0.1,1,10,100]

    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('C parameter: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        for iteration, indices in enumerate(fold.split(x_train_data)):

            # Call the logistic regression model with a certain C parameter
            lr = LogisticRegression(C = c_param, penalty = 'l1')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            lr.fit(x_train_data.iloc[indices[0],:],y_train_data.iloc[indices[0],:].values.ravel())

            # Predict values using the test indices in the training data
            y_pred_undersample = lr.predict(x_train_data.iloc[indices[1],:].values)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            recall_acc = recall_score(y_train_data.iloc[indices[1],:].values,y_pred_undersample)
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.ix[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c

polynomial
* 0.01 --> 0.76409
* 0.1 --> 0.7694
* 1 --> 0.76944

ploynomail + OHE

### K-FOLD  Validation

In [ ]:
best_c = printing_Kfold_scores(X_train, y_train)

In [ ]:
lr = LogisticRegression(C = best_c, penalty = 'l1')
lr.fit(X_train, y_train)
y_pred_sample = lr.predict(X_test.values)


In [ ]:
from sklearn import metrics
import seaborn as sns

cnf_matrix = metrics.confusion_matrix(y_test, y_pred_sample)
cnf_matrix

class_names=[0,1] # name  of classes

fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

### Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
# from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Dense(units=40,input_dim = 131 , kernel_initializer = 'uniform' , activation = 'relu'))
model.add(Dense(units=30 , kernel_initializer = 'uniform' , activation = 'relu'))
model.add(Dense(units=1 , kernel_initializer = 'uniform' , activation = 'sigmoid'))


In [ ]:
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# batch_size小苦於時間 batch_size大苦於空間
train_history = model.fit(x = X_train , y = y_train , validation_split = 0.1 , epochs=30 , batch_size =100 , verbose = 2)

In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

from sklearn.metrics import auc
auc_keras = auc(fpr_keras , tpr_keras)
auc_keras

In [ ]:
predictions = model.predict_classes(X_test).tolist()

def count_accuracy(yhat,y):
    length = len(yhat)
    count = 0
    y = list(y)
    for i in range(length):
        if yhat[i][0] == y[i]:
            count+=1
    accuracy = count/length
    return(accuracy)

count_accuracy(predictions , y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
predictions = model.predict_classes(X_test).tolist()
cm = confusion_matrix(y_test , predictions)
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True)


### XGB 參數

In [ ]:
# XGBoost
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

eval_set = [(X_train, y_train), (X_test, y_test)]

xgbc = XGBClassifier(eta = 0.15, gamma =0.2,  min_child_weight = 6 , max_depth =4 , subsample = 0.7 ,
                    n_estimators = 500, colsample_bytree = 0.7 , alpha = 0.8 , lambda_reg = 0.8, scoring = 'roc_auc', 
                     learning_rate = 0.1 , objective= 'binary:logistic', eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)

xgbc.fit(X_train, y_train)

y_pred_xgbc = xgbc.predict(X_test)

# retrieve performance metrics
results =xgbc.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()


#ans = list(xgbc.predict(test))


